In [1]:
using LinearAlgebra
using TensorKit
using JSON
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("load_tensor.jl")
include("CTM.jl")
#include("my_TensorKit_fun.jl")

#TensorKit.usebraidcache_nonabelian[] = false

D=12
chi=10
tol=1e-6


A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

#load elementary tensor coefficients from json file
function read_string(string)
    nums=ones(length(string))
    for cn=1:length(string)
        nums[cn]=parse.(Float64, split(string[cn])[2])
    end
    return nums
end
json_dict = Dict()
open("LS_D_"*string(D)*"_chi_40.json", "r") do f
    global json_dict
    dicttxt = read(f,String)  # file information to string
    json_dict=JSON.parse(dicttxt)  # parse and transform data
end

Bond_irrep=json_dict["Bond_irrep"]
Triangle_A1_coe=read_string(json_dict["coes"]["Triangle_A1_coe"]["entries"]);
Triangle_A2_coe=read_string(json_dict["coes"]["Triangle_A2_coe"]["entries"]);
if Bond_irrep=="A"
    Bond_A_coe=read_string(json_dict["coes"]["Bond_A_coe"]["entries"]);
elseif Bond_irrep=="B"
    Bond_B_coe=read_string(json_dict["coes"]["Bond_B_coe"]["entries"]);
elseif Bond_irrep=="A+iB"
    Bond_A_coe=read_string(json_dict["coes"]["Bond_A_coe"]["entries"]);
    Bond_B_coe=read_string(json_dict["coes"]["Bond_B_coe"]["entries"]);
end



global triangle_tensor=A1_set[1]*0;
global bond_tensor=A_set[1]*0;
if Bond_irrep=="A"
    global bond_tensor=A_set[1]*0;
    for ct=1:length(Bond_A_coe)
        global bond_tensor=bond_tensor+A_set[ct]*Bond_A_coe[ct]
    end
elseif Bond_irrep=="B"
    global bond_tensor=B_set[1]*0;
    for ct=1:length(Bond_B_coe)
        global bond_tensor=bond_tensor+B_set[ct]*Bond_B_coe[ct]
    end
elseif Bond_irrep=="A+iB"
    global bond_tensor=A_set[1]*0;
    for ct=1:length(Bond_A_coe)
        global bond_tensor=bond_tensor+A_set[ct]*Bond_A_coe[ct]
    end
    for ct=1:length(Bond_B_coe)
        global bond_tensor=bond_tensor+B_set[ct]*Bond_B_coe[ct]
    end
end

for ct=1:length(Triangle_A1_coe)
    global triangle_tensor=triangle_tensor+A1_set[ct]*Triangle_A1_coe[ct]
end
for ct=1:length(Triangle_A2_coe)
    global triangle_tensor=triangle_tensor+im*A2_set[ct]*Triangle_A2_coe[ct]
end

#TensorKit.usebraidcache_nonabelian[] = false
A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

#load elementary tensor coefficients from json file
function read_string(string)
    nums=ones(length(string))
    for cn=1:length(string)
        nums[cn]=parse.(Float64, split(string[cn])[2])
    end
    return nums
end
json_dict = Dict()
open("LS_D_"*string(D)*"_chi_40.json", "r") do f
    global json_dict
    dicttxt = read(f,String)  # file information to string
    json_dict=JSON.parse(dicttxt)  # parse and transform data
end

Bond_irrep=json_dict["Bond_irrep"]
Triangle_A1_coe=read_string(json_dict["coes"]["Triangle_A1_coe"]["entries"]);
Triangle_A2_coe=read_string(json_dict["coes"]["Triangle_A2_coe"]["entries"]);
if Bond_irrep=="A"
    Bond_A_coe=read_string(json_dict["coes"]["Bond_A_coe"]["entries"]);
elseif Bond_irrep=="B"
    Bond_B_coe=read_string(json_dict["coes"]["Bond_B_coe"]["entries"]);
elseif Bond_irrep=="A+iB"
    Bond_A_coe=read_string(json_dict["coes"]["Bond_A_coe"]["entries"]);
    Bond_B_coe=read_string(json_dict["coes"]["Bond_B_coe"]["entries"]);
end



global triangle_tensor=A1_set[1]*0;
global bond_tensor=A_set[1]*0;
if Bond_irrep=="A"
    global bond_tensor=A_set[1]*0;
    for ct=1:length(Bond_A_coe)
        global bond_tensor=bond_tensor+A_set[ct]*Bond_A_coe[ct]
    end
elseif Bond_irrep=="B"
    global bond_tensor=B_set[1]*0;
    for ct=1:length(Bond_B_coe)
        global bond_tensor=bond_tensor+B_set[ct]*Bond_B_coe[ct]
    end
elseif Bond_irrep=="A+iB"
    global bond_tensor=A_set[1]*0;
    for ct=1:length(Bond_A_coe)
        global bond_tensor=bond_tensor+A_set[ct]*Bond_A_coe[ct]
    end
    for ct=1:length(Bond_B_coe)
        global bond_tensor=bond_tensor+B_set[ct]*Bond_B_coe[ct]
    end
end

for ct=1:length(Triangle_A1_coe)
    global triangle_tensor=triangle_tensor+A1_set[ct]*Triangle_A1_coe[ct]
end
for ct=1:length(Triangle_A2_coe)
    global triangle_tensor=triangle_tensor+im*A2_set[ct]*Triangle_A2_coe[ct]
end

@time PEPS_tensor=bond_tensor
@time @tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];

# display(codomain(PEPS_tensor))
# @time PEPS_tensor1= ncon([bond_tensor,bond_tensor,bond_tensor,triangle_tensor,triangle_tensor],[[-1,1,-5],[4,3,-6],[-4,2,-7],[1,3,2],[4,-2,-3]]);

# @time @tensoropt PEPS_tensor2[-1,-2,-3,-4,-5,-6,-7] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];

# @time @tensoropt PEPS_tensor3[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];


@time U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@time @tensor PEPS_tensor[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];


# #PEPS_tensor_dense=convert(Array,PEPS_tensor)

# #save initial CTM to compare with other codes
# @time CTM=init_CTM(10,PEPS_tensor,"PBC");
# matwrite("matfile.mat", Dict(
# 	"C1" => convert(Array,CTM["Cset"][1]),
# 	"C2" => convert(Array,CTM["Cset"][2]),
#     "C3" => convert(Array,CTM["Cset"][3]),
#     "C4" => convert(Array,CTM["Cset"][4]),
#     "T1" => convert(Array,CTM["Tset"][1]),
#     "T2" => convert(Array,CTM["Tset"][2]),
#     "T3" => convert(Array,CTM["Tset"][3]),
#     "T4" => convert(Array,CTM["Tset"][4])
# ); compress = false)

type="PBC"
A=PEPS_tensor
display("initialize CTM")
#numind(A)
#numin(A)
#numout(A)
CTM=[]
Cset=Vector(undef,4)
Tset=Vector(undef,4)
#space(A,1)
@time if type=="PBC"
    for direction=1:4    
        inds=(mod1(2-direction,4),mod1(3-direction,4),mod1(4-direction,4),mod1(1-direction,4),5);
        A_rotate=permute(A,inds)
        Ap_rotate=A_rotate'

        @tensor M[:]:=Ap_rotate[1,-1,-3,2,3]*A_rotate[1,-2,-4,2,3];
        Cset[direction]=M;
        @tensor M[:]:=Ap_rotate[-1,-3,-5,1,2]*A_rotate[-2,-4,-6,1,2];
        Tset[direction]=M
    end

    #fuse legs
    ul_set=Vector(undef,4)
    ur_set=Vector(undef,4)
    for direction=1:2
        ul_set[direction]=unitary(fuse(space(Cset[direction], 3) ⊗ space(Cset[direction], 4)), space(Cset[direction], 3) ⊗ space(Cset[direction], 4))
        ur_set[direction]=unitary(fuse(space(Tset[direction], 5) ⊗ space(Tset[direction], 6)), space(Tset[direction], 5) ⊗ space(Tset[direction], 6))
    end
    for direction=3:4
        ul_set[direction]=unitary(fuse(space(Cset[direction], 3) ⊗ space(Cset[direction], 4))', space(Cset[direction], 3) ⊗ space(Cset[direction], 4))
        ur_set[direction]=unitary(fuse(space(Tset[direction], 5) ⊗ space(Tset[direction], 6))', space(Tset[direction], 5) ⊗ space(Tset[direction], 6))
    end
    for direction=1:4
        C=Cset[direction]
        ul=ur_set[mod1(direction-1,4)]
        ur=ul_set[direction]
        ulp=permute(ul',(3,),(1,2,))
        urp=permute(ur',(3,),(1,2,))
        #@tensor Cnew[(-1);(-2)]:=ulp[-1,1,2]*C[1,2,3,4]*ur[-2,3,4]
        @tensor Cnew[:]:=ulp[-1,1,2]*C[1,2,3,4]*ur[-2,3,4]#put all indices in tone side so that its adjoint has the same index order 
        Cset[direction]=Cnew

        T=Tset[direction]
        ul=ul_set[direction]
        ur=ur_set[direction]
        ulp=permute(ul',(3,),(1,2,))
        urp=permute(ur',(3,),(1,2,))
        #@tensor Tnew[(-1);(-2,-3,-4)]:=ulp[-1,1,2]*T[1,2,-2,-3,3,4]*ur[-4,3,4]
        @tensor Tnew[:]:=ulp[-1,1,2]*T[1,2,-2,-3,3,4]*ur[-4,3,4]#put all indices in tone side so that its adjoint has the same index order 
        Tset[direction]=Tnew
    end
elseif type=="random"
end
CTM=Dict([("Cset", Cset), ("Tset", Tset)])

# matwrite("matfile.mat", Dict(
# 	"C1" => convert(Array,CTM["Cset"][1]),
# 	"C2" => convert(Array,CTM["Cset"][2]),
#     "C3" => convert(Array,CTM["Cset"][3]),
#     "C4" => convert(Array,CTM["Cset"][4]),
#     "T1" => convert(Array,CTM["Tset"][1]),
#     "T2" => convert(Array,CTM["Tset"][2]),
#     "T3" => convert(Array,CTM["Tset"][3]),
#     "T4" => convert(Array,CTM["Tset"][4])
# ); compress = false)


"initialize CTM"

  0.000001 seconds
 30.680206 seconds (111.99 M allocations: 5.866 GiB, 7.83% gc time, 98.84% compilation time)
  0.233022 seconds (348.64 k allocations: 18.456 MiB, 99.92% compilation time)
  5.586457 seconds (21.33 M allocations: 1.429 GiB, 7.92% gc time, 85.14% compilation time)
 30.039941 seconds (126.98 M allocations: 11.159 GiB, 11.89% gc time, 64.18% compilation time)


Dict{String, Vector{Any}} with 2 entries:
  "Cset" => [TensorMap((Rep[SU₂](0=>7, 1/2=>10, 1=>12, 3/2=>8, 2=>6, 5/2=>2, 3=…
  "Tset" => [TensorMap((Rep[SU₂](0=>7, 1/2=>10, 1=>12, 3/2=>8, 2=>6, 5/2=>2, 3=…

In [2]:
display(domain(A))
display(codomain(A))
@time A=permute(A,(5,),(1,2,3,4,));


ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

(Rep[SU₂](0=>1, 1/2=>2, 1=>1, 3/2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>2, 1=>1, 3/2=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>2, 1=>1, 3/2=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>2, 1=>1, 3/2=>1) ⊗ Rep[SU₂](1/2=>2, 3/2=>1))

  1.126821 seconds (3.00 M allocations: 199.624 MiB, 4.28% gc time, 92.49% compilation time)


In [3]:

@time double_layer=A'*A;


  0.287854 seconds (353.27 k allocations: 68.352 MiB, 15.76% gc time, 77.11% compilation time)


In [4]:
@time U_L=unitary(fuse(space(double_layer, 1) ⊗ space(double_layer, 5)), space(double_layer, 1) ⊗ space(double_layer, 5));


  0.007965 seconds (6.56 k allocations: 374.208 KiB, 98.50% compilation time)


In [ ]:

@time @tensor double_layer[:] :=double_layer[1,-2,-3,-4,2,-5,-6,-7]*U_L[-1,1,2];


In [ ]:

#@time double_layer=permute(double_layer,(1,5,2,6,3,7,4,8),())


In [ ]:
@time double_layer=permute(double_layer,(1,5,2,3,4,),(6,7,8,));


In [ ]:
@time U_L=unitary(fuse(space(double_layer, 1) ⊗ space(double_layer, 2)), space(double_layer, 1) ⊗ space(double_layer, 2));


In [ ]:

@time @tensor double_layer[:] :=double_layer[1,2,-2,-3,-4,-5,-6,-7]*U_L[-1,1,2];



In [ ]:
double_layer=permute(double_layer,(1,2,),(3,4,5,6,7,8));

In [ ]:

@time double_layer=U_L*double_layer;



In [ ]:
@time double_layer=permute(double_layer,(2,5,),(1,3,4,6,7));


In [ ]:
@time U_d=unitary(fuse(space(double_layer, 1) ⊗ space(double_layer, 2)), space(double_layer, 1) ⊗ space(double_layer, 2));


In [ ]:
@time double_layer=U_d*double_layer;



In [ ]:
@time double_layer=permute(double_layer,(3,5,),(2,1,4,6,));


In [9]:
@time U_R=U_L';
display(space(UR,1))
display(space(UR,2))
display(space(UR,3))


Rep[SU₂](0=>1, 1/2=>2, 1=>1, 3/2=>1)'

Rep[SU₂](0=>1, 1/2=>2, 1=>1, 3/2=>1)

Rep[SU₂](0=>7, 1/2=>10, 1=>12, 3/2=>8, 2=>6, 5/2=>2, 3=>1)'

  0.000010 seconds (1 allocation: 16 bytes)


In [10]:
@time U_R=permute(U_R,(3,),(1,2,));


  0.005533 seconds (18.43 k allocations: 1.443 MiB, 85.70% compilation time)


In [ ]:
@time double_layer=U_R*double_layer;

In [ ]:


    #Ref: PHYSICAL REVIEW B 98, 235148 (2018)

    #initial corner transfer matrix

    Cset=CTM["Cset"];
    Tset=CTM["Tset"];
    conv_check="singular_value"

    ss_old=ones(chi)*2;
    d=2;
    rho_old=Matrix(I,d^3,d^3);

    #Iteration
    CTM_ite_nums=50;
    display("start CTM iterations:")
    @time for ci=1:1#CTM_ite_nums
        #told=Tset{4};
        #pold=Pset{4};
        direction_order=[1,2,3,4];
        for direction=1# in direction_order
            #disp(['direction ', num2str(direction)])
            
            
            #direction=1, 2, 3, 4:  left, up, right, down move

            A=permute(A, (mod1(2-direction,4),mod1(3-direction,4),mod1(4-direction,4),mod1(1-direction,4),),(5,));
            
            M1=Cset[mod1(direction,4)];#mod1(direction,4)
            M2=Tset[mod1(direction,4)];#mod1(direction,4)
            M3=Tset[mod1(direction-1,4)];#mod1(direction-1,4)
            M4=permute(A,(1,2,3,4,5,),());
            M5=Tset[mod1(direction-1,4)];#mod1(direction-1,4)
            M6=permute(A,(1,2,3,4,5,),());
            M7=Cset[mod1(direction-1,4)];#mod1(direction-1,4)
            M8=Tset[mod1(direction-2,4)];#mod1(direction-2,4)
            
            M1_reflect=Cset[mod1(direction+1,4)];
            M2_reflect=Tset[mod1(direction,4)];
            M3_reflect=Tset[mod1(direction+1,4)];
            M4_reflect=permute(A,(1,2,3,4,5,),());
            M5_reflect=Tset[mod1(direction+1,4)];
            M6_reflect=permute(A,(1,2,3,4,5,),());
            M7_reflect=Cset[mod1(direction-2,4)];
            M8_reflect=Tset[mod1(direction-2,4)];
            
            @tensor M3M4[:]:=M3[-1,1,2,-6]* M4'[1,-2,-4,-7,3]* M4[2,-3,-5,-8,3];  
            # #M3M4=reshape(M3M4,[chi*D^2,D,D,chi*D^2]);
            # @tensor M5M6[:]:=M5[-1,1,2,-6]* M6'[1,-2,-4,-7,3]* M6[2,-3,-5,-8,3];  
            # #M5M6=reshape(M5M6,[chi*D^2,D,D,chi*D^2]);
            # @tensor M1M2[:]:=M1[-1,1]* M2[1,-2,-3,-4];  
            # #M1M2=reshape(M1M2, [chi*D^2, chi]);
            # @tensor M7M8[:]:=M7[1,-2]* M8[-1,-3,-4,1];  
            # #M7M8=reshape(M7M8, [chi, chi*D^2]);
            # @tensor M1M2[:]:=M1[-1,1]* M2[1,-2,-3,-4];  
            # #M1M2=reshape(M1M2, [chi*D^2, chi]);
            # @tensor M7M8[:]:=M7[1,-2]* M8[-1,-3,-4,1];  
            # #M7M8=reshape(M7M8, [chi, chi*D^2]);

            # @tensor MMup[:]:=M1M2[1,2,3,-4]* M3M4[-1,-2,-3,-5,-6,1,2,3]; 
            # MMup=permute(MMup,(1,2,3,),(4,5,6,))
            # #MMup=reshape(MMup, [chi*D^2,chi*D^2]);
            # @tensor MMlow[:]:=M7M8[-4,1,2,3]* M5M6[1,2,3,-5,-6,-1,-2,-3]; 
            # MMlow=permute(MMlow,(1,2,3,),(4,5,6,))
            # #MMlow=reshape(MMlow, [chi*D^2,chi*D^2]);

            # @tensor MMup_reflect[:]:=M2_reflect[-1,3,4,1]* M1_reflect[1,2]* M4_reflect'[-2,-5,5,3,7]* M4_reflect[-3,-6,6,4,7]* M3_reflect[2,5,6,-4]; 
            # MMup_reflect=permute(MMup_reflect,(1,2,3,),(4,5,6,))
            # #MMup_reflect=reshape(MMup_reflect,[chi*D^2,chi*D^2]);
            # @tensor MMlow_reflect[:]:=M8_reflect[1,3,4,-1]* M7_reflect[2,1]* M6_reflect'[-2,3,5,-5,7]* M6_reflect[-3,4,6,-6,7]* M5_reflect[-4,5,6,2]; 
            # MMlow_reflect=permute(MMlow_reflect,(1,2,3,),(4,5,6,))
            # #MMlow_reflect=reshape(MMlow_reflect,[chi*D^2,chi*D^2]);
            # _, RMup=leftorth(permute(MMup*MMup_reflect,(4,5,6,),(1,2,3,)));
            # _, RMlow=leftorth(permute(MMlow*MMlow_reflect,(4,5,6,),(1,2,3,))); 

            # RMlow=permute(RMlow,(2,3,4,),(1,));
            
            # M=RMup*RMlow;
            # uM,sM,vM = tsvd(M; trunc=truncdim(chi));
            # #uM,sM,vM = tsvd(M; trunc=trunbelow(1e-34)); #this looks to truncate absolute value, not relative value
            # #display(convert(Array,sM))
            # #display(convert(Array,inv(sM)))
            # sM=sM/norm(sM)
            # sM_inv=inv(sM);
            # sM_dense=convert(Array,sM)
            # #display(sM_dense)
            # trun_tol=1e-12
            # for c1=1:size(sM_dense,1)
            #     if sM_dense[c1,c1]<trun_tol
            #         sM_dense[c1,c1]=0;
            #     end
            # end
            # #display(sM_dense)
            # #display(pinv.(sM_dense))

            
            #display(sM_inv)
            #display(convert(Array,sM_inv))
            #sM_inv_sqrt=sqrt.(convert(Array,sM_inv))
            #display(space(sM_inv))
            #display(sM_inv_sqrt)
            # sM_inv_sqrt=TensorMap(pinv.(sqrt.(sM_dense)),codomain(sM_inv)←domain(sM_inv))

            # PM_inv=RMlow*vM'*sM_inv_sqrt; 
            # PM=sM_inv_sqrt*uM'*RMup; 
            # PM=permute(PM,(2,3,4,),(1,));
            
            # @tensor M5tem[:]:=M3M4[1,2,3,-2,-3,4,5,6]* PM_inv[1,2,3,-1]* PM[4,5,6,-4];
            # @tensor M1tem[:]:=M1M2[1,2,3,-2]* PM_inv[1,2,3,-1];
            # @tensor M7tem[:]:=M7M8[-1,1,2,3]* PM[1,2,3,-2];
            
            # Cset[mod1(direction,4)]=M1tem/norm(M1tem);
            # Tset[mod1(direction-1,4)]=M5tem/norm(M5tem);
            # Cset[mod1(direction-1,4)]=M7tem/norm(M7tem);
            




        end

        if conv_check=="singular_value" #check convergence of singular value
            _,ss_new,_=svd(permute(Cset[1],(1,),(2,)));
            ss_new=convert(Array,ss_new);
            ss_new=sort(diag(ss_new), rev=true);
            ss_old=ss_old/ss_old[1];
            ss_new=ss_new/ss_new[1];
            #display(ss_new)
            if length(ss_old)>length(ss_new)
                dss=copy(ss_old);
                siz=length(ss_new)
            elseif length(ss_old)<=length(ss_new)
                dss=copy(ss_new);
                siz=length(ss_old)
            end
            dss[1:siz]=ss_old[1:siz]-ss_new[1:siz]
            er=norm(dss);
            display("CTMRG iteration: "*string(ci)*", CTMRG err: "*string(er));
            if er<tol
                break;
            end
            ss_old=ss_new;
        elseif conv_check=="density_matrix" #check reduced density matrix
            
            # ob_opts.SiteNumber=1;
            # CTM_tem.Cset=Cset;
            # CTM_tem.Tset=Tset;
            # rho_new=ob_CTMRG(CTM_tem,A,ob_opts).A;
            # er=sum(sum((abs(rho_old-rho_new))));
            # disp(['CTMRG iteration: ',num2str(ci),' CTMRG err: ',num2str(er)]);
            # if er<tol
            #     break;
            # end
            # rho_old=rho_new;
        end
        
        # if ci==CTM_ite_nums
        #     display(er)
        #     warn("CTMRG does not converge: " * string(er));
        # end
    end





